# Import Necessary Libraries

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train_new.csv')
train = train.sample(n=10000).reset_index(drop=True)
train.head()

,image,class
0,v_PushUps_g25_c04.avi_frame0.jpg,PushUps
1,v_TaiChi_g08_c02.avi_frame2.jpg,TaiChi
2,v_Rafting_g13_c02.avi_frame0.jpg,Rafting
3,v_BoxingPunchingBag_g25_c05.avi_frame1.jpg,BoxingPunchingBag
4,v_BoxingSpeedBag_g09_c07.avi_frame11.jpg,BoxingSpeedBag


In [3]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:45<00:00, 222.12it/s]


(10000, 224, 224, 3)

In [4]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [5]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [6]:
# creating the base model of pre-trained VGG16 model
base_model = MobileNetV2(weights='imagenet', include_top=False)

9412608/9406464 [==============================] - 0s 0us/step


In [7]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(8000, 7, 7, 1280)

In [8]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(2000, 7, 7, 1280)

In [10]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(8000, 7*7*1280)
X_test = X_test.reshape(2000, 7*7*1280)

In [11]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [12]:
# shape of images
X_train.shape

(8000, 62720)

In [17]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(62720,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))

In [18]:
# defining a function to save the weights of best model
from tensorflow.keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('MobileNetV2weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [19]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [20]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/200
63/63 [==============================] - 14s 224ms/step - loss: 4.8320 - accuracy: 0.0191 - val_loss: 4.5156 - val_accuracy: 0.0620
Epoch 2/200
63/63 [==============================] - 15s 232ms/step - loss: 4.4762 - accuracy: 0.0417 - val_loss: 4.3076 - val_accuracy: 0.0665
Epoch 3/200
63/63 [==============================] - 15s 243ms/step - loss: 4.2611 - accuracy: 0.0739 - val_loss: 3.9593 - val_accuracy: 0.1655
Epoch 4/200
63/63 [==============================] - 15s 240ms/step - loss: 3.9694 - accuracy: 0.1204 - val_loss: 3.5197 - val_accuracy: 0.2455
Epoch 5/200
63/63 [==============================] - 15s 245ms/step - loss: 3.6609 - accuracy: 0.1733 - val_loss: 3.1677 - val_accuracy: 0.2905
Epoch 6/200
63/63 [==============================] - 15s 243ms/step - loss: 3.3902 - accuracy: 0.2170 - val_loss: 2.9347 - val_accuracy: 0.3505
Epoch 7/200
63/63 [==============================] - 15s 240ms/step - loss: 3.1732 - accuracy: 0.2549 - val_loss: 2.7129 - val_accuracy:

63/63 [==============================] - 14s 216ms/step - loss: 1.0535 - accuracy: 0.7051 - val_loss: 0.9999 - val_accuracy: 0.7600
Epoch 114/200
63/63 [==============================] - 15s 230ms/step - loss: 1.0627 - accuracy: 0.7031 - val_loss: 0.9719 - val_accuracy: 0.7695
Epoch 115/200
63/63 [==============================] - 14s 216ms/step - loss: 1.0421 - accuracy: 0.7096 - val_loss: 0.9798 - val_accuracy: 0.7620
Epoch 116/200
63/63 [==============================] - 14s 216ms/step - loss: 1.0499 - accuracy: 0.7114 - val_loss: 1.0054 - val_accuracy: 0.7565
Epoch 117/200
63/63 [==============================] - 14s 216ms/step - loss: 1.0454 - accuracy: 0.7080 - val_loss: 1.0120 - val_accuracy: 0.7510
Epoch 118/200
63/63 [==============================] - 14s 216ms/step - loss: 1.0611 - accuracy: 0.7044 - val_loss: 0.9818 - val_accuracy: 0.7560
Epoch 119/200
63/63 [==============================] - 14s 230ms/step - loss: 1.0444 - accuracy: 0.7097 - val_loss: 0.9716 - val_accuracy:

In [21]:
# loading the trained weights
model.load_weights("MobileNetV2weight.hdf5")

In [22]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [23]:
# getting the test list
f = open("ucfTrainTestlist/testlist03.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c05.avi


In [24]:
# creating the tags
train = pd.read_csv('train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [25]:
import cv2
import math
import os
from glob import glob
from scipy import stats as s

In [27]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('UCF-101/'+videoFile.split(' ')[0])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
            filename ='temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*1280)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(videoFile.split('/')[1].split('_')[1])

  0%|                                                                                         | 0/3696 [00:00<?, ?it/s]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


100%|██████████████████████████████████████████████████████████████████████████████| 3696/3696 [15:44<00:00,  3.91it/s]


In [28]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

74.54004329004329

# Model Evaluation

Model accuracy is significantly better, and overall training time is shorter. The MobileNetV2 can be one of the considerations for training on action recognition based on limited training data. 